### stage_model_classifier

base: 14./stage_model_classifier_withneighbor-newfeatures

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
#classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
classifiers = ['currank','avgrank','dice','lr','lsvc','lsvcl2','rf','lrbias','xgb']
train_x, train_y, test_x, test_y = None, None, None, None

def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
def test_20182019():
    global train_x, train_y, test_x, test_y
    
    ### fix train
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(classifiers)
    print('cols:%s'%cols)
    retdf = pd.DataFrame([],columns=cols)


    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}

    #first 
    eventid = events_id['Indy500-2018']
    ignore_eventid = events_id['Indy500-2019']
    stdata_2018 = stagedata[stagedata['eventid']!=ignore_eventid]

    print('Testset = %s'%eventsname[eventid])

    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stdata_2018, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)

    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        


    eventid = events_id['Indy500-2019']
    ignore_eventid = events_id['Indy500-2018']
    stdata_2019 = stagedata[stagedata['eventid']!=ignore_eventid]

    print('Testset = %s'%eventsname[eventid])

    train2, test2, train_x2, train_y2, test_x, test_y = split_by_eventid(stdata_2019, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))

    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)

    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df]) 
    
    retdf.to_csv(f'stint_classifier_result_{suffix}.csv', float_format='%.3f')
    return retdf

def test_cv():
    global train_x, train_y, test_x, test_y
    
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(classifiers)
    print('cols:%s'%cols)
    retdf = pd.DataFrame([],columns=cols)


    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}


    for eventid in events:
        print('Testset = %s'%eventsname[eventid])

        train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        #record
        rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

        acc = [0 for x in range(len(classifiers))]
        for idx, clf in enumerate(classifiers):
            acc[idx] = classifier_model(clf)

        rec.extend(acc)
        print('rec:%s'%rec)

        #new df
        df = pd.DataFrame([rec],columns=cols)
        retdf = pd.concat([retdf, df])        

    retdf.to_csv('crossvalid_stagedata_classifier_%s.csv'%suffix, float_format='%.3f')
    #df_event = retdf
    return retdf

### test oracle with stintlen

In [5]:
#load data
_trim = 0
_include_final = True
_include_stintlen = True
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
suffix = f'indy500-2013-2019-end{include_str}{stint_str}-t{_trim}'
output_file = f'shortterm-indy500-2013-2019-end{include_str}{stint_str}-t{_trim}.csv'
stagedata = pd.read_csv(output_file)

stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31487 entries, 0 to 31486
Data columns (total 37 columns):
Unnamed: 0                   31487 non-null int64
target                       31487 non-null int64
start_lap                    31487 non-null int64
stint_len                    31487 non-null int64
eventid                      31487 non-null int64
car_number                   31487 non-null int64
stageid                      31487 non-null int64
firststage                   31487 non-null int64
pit_in_caution               31487 non-null int64
start_position               31487 non-null int64
start_rank                   31487 non-null int64
start_rank_ratio             31487 non-null float64
top_pack                     31487 non-null int64
bottom_pack                  31487 non-null int64
average_rank                 31487 non-null float64
average_rank_all             31487 non-null float64
change_in_rank               31487 non-null int64
change_in_rank_all           31487 

In [6]:
stagedata.head(5)

,Unnamed: 0,target,start_lap,stint_len,eventid,car_number,stageid,firststage,pit_in_caution,start_position,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,0,42,0,1,0,0,0,3,...,0.000000,0,0,0.0000,0,0,0,0,0,0
1,1,0,1,42,0,1,1,1,0,3,...,25.236722,40,10,62.0677,0,0,0,0,0,0
2,2,0,2,42,0,1,2,1,0,3,...,25.771546,40,11,62.0677,0,0,0,0,0,0
3,3,0,3,42,0,1,3,1,0,3,...,26.200225,40,12,62.0677,0,0,0,0,0,0
4,4,0,4,42,0,1,4,1,0,3,...,26.110915,40,13,62.0677,0,0,0,0,0,0


In [7]:
df_event = test_cv()

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2013
[*] predict with currank model
precision=[0.         0.14871795 0.        ], recall=[0. 1. 0.], f1=[0.         0.25892857 0.        ], accuracy=0.15
[*] predict with avgrank model
precision=[0.40425532 0.33333333 0.22972973], recall=[0.3877551  0.31034483 0.25      ], f1=[0.39583333 0.32142857 0.23943662], accuracy=0.33
[*] predict with dice model
precision=[0.46902655 0.2173913  0.27118644], recall=[0.54081633 0.17241379 0.23529412], f1=[0.50236967 0.19230769 0.2519685 ], accuracy=0.38
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.61538462 0.5        0.6031746 ], recall=[0.81632653 0.03448276 0.55882353], f1=[0.70175439 0.06451613 0.58015267], accuracy=0.61
[*] predict with lsvc model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.625 0.5   0.6  ], recall=[0.81632653 0.03448276 0.57352941], f1=[0.7079646  0.06451613 0.58646617], accuracy=0.62
[*] predict with lsvcl2 model


...

precision=[0.65384615 0.66666667 0.51807229], recall=[0.72649573 0.06060606 0.65151515], f1=[0.68825911 0.11111111 0.57718121], accuracy=0.60
[*] predict with lsvcl2 model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.64705882 0.33333333 0.625     ], recall=[0.84615385 0.15151515 0.45454545], f1=[0.73333333 0.20833333 0.52631579], accuracy=0.62
[*] predict with rf model
precision=[0.63414634 0.66666667 0.51724138], recall=[0.66666667 0.12121212 0.68181818], f1=[0.65       0.20512821 0.58823529], accuracy=0.59
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


precision=[0.66412214 0.5        0.53246753], recall=[0.74358974 0.12121212 0.62121212], f1=[0.7016129  0.19512195 0.57342657], accuracy=0.61
[*] predict with xgb model
precision=[0.63909774 0.5        0.53623188], recall=[0.72649573 0.21212121 0.56060606], f1=[0.68       0.29787234 0.54814815], accuracy=0.60
rec:['Indy500-2019', 1306, 216, '+:66,0:33,-:117', 0.1527777777777778, 0.37037037037037035, 0.4861111111111111, 0.6157407407407407, 0.6018518518518519, 0.6203703703703703, 0.5879629629629629, 0.6111111111111112, 0.5972222222222222]


In [7]:
retdf_oracle = test_20182019()

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2018
[*] predict with currank model
precision=[0.         0.54255076 0.        ], recall=[0. 1. 0.], f1=[0.         0.70344624 0.        ], accuracy=0.54
[*] predict with avgrank model
precision=[0.         0.         0.11498973], recall=[0. 0. 1.], f1=[0.         0.         0.20626151], accuracy=0.11
[*] predict with dice model
precision=[0.33885667 0.54286843 0.1255814 ], recall=[0.27248501 0.57779647 0.16071429], f1=[0.30206795 0.55978814 0.14099217], accuracy=0.43
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.40909091 0.5677675  0.38023952], recall=[0.07195203 0.90370059 0.25198413], f1=[0.1223796  0.69738764 0.30310263], accuracy=0.54
[*] predict with lsvc model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.408      0.55455868 0.66666667], recall=[0.0679547  0.96173255 0.01190476], f1=[0.11650485 0.70347585 0.02339181], accuracy=0.55
[*] predict with lsvcl2 model


...

precision=[0.45454545 0.65085432 0.        ], recall=[0.09319287 0.96725693 0.        ], f1=[0.15467384 0.77812122 0.        ], accuracy=0.64
[*] predict with lsvcl2 model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.         0.63667305 0.        ], recall=[0. 1. 0.], f1=[0.         0.77800884 0.        ], accuracy=0.64
[*] predict with rf model
precision=[0.35765896 0.72582287 0.41621622], recall=[0.40113452 0.71466756 0.32489451], f1=[0.37815126 0.72020202 0.36492891], accuracy=0.59
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished


precision=[0.41366906 0.64961169 0.62222222], recall=[0.09319287 0.95021717 0.05907173], f1=[0.1521164  0.77167277 0.10789981], accuracy=0.64
[*] predict with xgb model
precision=[0.32323997 0.70531915 0.325     ], recall=[0.34602917 0.66455062 0.38396624], f1=[0.33424658 0.68432823 0.35203095], accuracy=0.55
rec:['Indy500-2019', 22403, 4701, '+:474,0:2993,-:1234', 0.6366730482875984, 0.10082961072112316, 0.45756222080408426, 0.6364603275898745, 0.6402893001489045, 0.6366730482875984, 0.5930653052542012, 0.635396724101255, 0.5526483726866624]


In [8]:
retdf_oracle

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2018,22403,4383,"+:504,0:2378,-:1501",0.542551,0.11499,0.425279,0.54392,0.546429,0.372348,0.546429,0.545289,0.510381
0,Indy500-2019,22403,4701,"+:474,0:2993,-:1234",0.636673,0.10083,0.457562,0.63646,0.640289,0.636673,0.593065,0.635397,0.552648


In [11]:
events = set(stagedata['eventid'])

years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
eventsname = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(eventsname)}
eventid = events_id['Indy500-2018']
ignore_eventid = events_id['Indy500-2019']
stdata_2018 = stagedata[stagedata['eventid']!=ignore_eventid]
test_2018 =   stagedata[stagedata['eventid']==events_id['Indy500-2018']]

In [12]:
test_2018

,Unnamed: 0,target,start_lap,stint_len,eventid,car_number,stageid,firststage,pit_in_caution,start_position,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
22403,22403,0,0,42,5,1,0,0,0,3,...,0.000000,0,0,0.00000,0,0,0,0,0,0
22404,22404,0,1,42,5,1,1,1,0,3,...,8.567336,40,9,61.12735,0,0,0,0,0,0
22405,22405,0,2,42,5,1,2,1,0,3,...,8.473138,40,10,61.12735,0,0,0,0,0,0
22406,22406,0,3,42,5,1,3,1,0,3,...,8.380925,40,11,61.12735,0,0,0,0,0,0
22407,22407,0,4,42,5,1,4,1,0,3,...,8.291206,40,12,61.12735,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26781,26781,-2,154,42,5,98,154,1,0,10,...,22.591369,40,24,59.99830,0,0,0,0,0,0
26782,26782,-4,155,42,5,98,155,1,0,10,...,22.545040,40,25,59.99830,0,0,0,0,0,0
26783,26783,-2,156,42,5,98,156,1,0,10,...,22.501341,40,26,59.99830,0,0,0,0,0,0
26784,26784,0,157,42,5,98,157,1,0,10,...,22.460143,40,27,59.99830,0,0,0,0,0,0


In [13]:
len(test_2018[test_2018['target']==0])

2378

In [14]:
test_2018[test_2018['car_number']==12]

,Unnamed: 0,target,start_lap,stint_len,eventid,car_number,stageid,firststage,pit_in_caution,start_position,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
23312,23312,0,0,42,5,12,0,0,0,6,...,0.000000,0,0,0.00000,0,0,0,0,0,0
23313,23313,0,1,42,5,12,1,1,0,6,...,8.716711,40,9,61.85245,0,0,0,0,0,0
23314,23314,0,2,42,5,12,2,1,0,6,...,8.619699,40,10,61.85245,0,0,0,0,0,0
23315,23315,0,3,42,5,12,3,1,0,6,...,8.527553,40,11,61.85245,0,0,0,0,0,0
23316,23316,0,4,42,5,12,4,1,0,6,...,8.437718,40,12,61.85245,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23466,23466,-2,154,42,5,12,154,1,0,6,...,23.131084,40,23,58.78955,0,0,0,0,0,0
23467,23467,-2,155,42,5,12,155,1,0,6,...,23.087297,40,24,58.78955,0,0,0,0,0,0
23468,23468,0,156,42,5,12,156,1,0,6,...,23.049108,40,25,58.78955,0,0,0,0,0,0
23469,23469,0,157,42,5,12,157,1,0,6,...,23.008583,40,26,58.78955,0,0,0,0,0,0
